In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
# Hyper-parameters 
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

100.1%

Extracting ../../data\MNIST\raw\train-images-idx3-ubyte.gz to ../../data\MNIST\raw


113.5%

Extracting ../../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../../data\MNIST\raw


100.4%

Extracting ../../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data\MNIST\raw


180.4%C:\Users\5001\anaconda3\envs\uda\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting ../../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data\MNIST\raw
Processing...
Done!


In [4]:
# Logistic regression model
model = nn.Linear(input_size, num_classes)

In [5]:
# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)

In [6]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2290
Epoch [1/5], Step [200/600], Loss: 2.1476
Epoch [1/5], Step [300/600], Loss: 2.0629
Epoch [1/5], Step [400/600], Loss: 1.9169
Epoch [1/5], Step [500/600], Loss: 1.9190
Epoch [1/5], Step [600/600], Loss: 1.8280
Epoch [2/5], Step [100/600], Loss: 1.7164
Epoch [2/5], Step [200/600], Loss: 1.7195
Epoch [2/5], Step [300/600], Loss: 1.5837
Epoch [2/5], Step [400/600], Loss: 1.5455
Epoch [2/5], Step [500/600], Loss: 1.5261
Epoch [2/5], Step [600/600], Loss: 1.4707
Epoch [3/5], Step [100/600], Loss: 1.4131
Epoch [3/5], Step [200/600], Loss: 1.4782
Epoch [3/5], Step [300/600], Loss: 1.4135
Epoch [3/5], Step [400/600], Loss: 1.2552
Epoch [3/5], Step [500/600], Loss: 1.3237
Epoch [3/5], Step [600/600], Loss: 1.3260
Epoch [4/5], Step [100/600], Loss: 1.3087
Epoch [4/5], Step [200/600], Loss: 1.1180
Epoch [4/5], Step [300/600], Loss: 1.1819
Epoch [4/5], Step [400/600], Loss: 1.2140
Epoch [4/5], Step [500/600], Loss: 1.1783
Epoch [4/5], Step [600/600], Loss:

In [9]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct //total))


Accuracy of the model on the 10000 test images: 82 %


In [8]:

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')